<a href="https://colab.research.google.com/github/y0o0ic/AIS_Visualization/blob/master/AIS_big_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データのダウンロード　（1年の全体容量97GB：半年分をダウンロード）

In [0]:
# ! wget -np -r -nH -L --cut-dirs=3 https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2018/

In [0]:
import os

In [0]:
dirs = os.listdir("2018/")

In [247]:
dirs[0:10]

['AIS_2018_02_10.zip',
 'AIS_2018_02_24.zip',
 'AIS_2018_05_04.zip',
 'AIS_2018_02_11.zip',
 'AIS_2018_04_19.zip',
 'AIS_2018_04_21.zip',
 'AIS_2018_05_15.zip',
 'AIS_2018_04_11.zip',
 'AIS_2018_04_08.zip',
 'AIS_2018_02_02.zip']

In [0]:
# ! unzip "2018/AIS_2018_01_*.zip"

In [0]:
# ! rm -r 2018/AIS_2018_01_*.zip

In [0]:
import glob

In [0]:
csvs = glob.glob("*.csv")

In [0]:
csvs.sort()

In [0]:
import pandas as pd
import numpy as np

In [0]:
# rdf = pd.read_csv(csvs[0])

In [0]:
# rdf['datetime'] = pd.to_datetime(rdf['BaseDateTime'])
# rdf.dropna(subset=['VesselType'],inplace=True)
# rdf["iVesselType"]=rdf["VesselType"].astype(np.int64)
# rdf.query('VesselType in [31]',inplace=True)
# # rdf.query('VesselType in @iships',inplace=True)

In [0]:
# rdf.head()

In [0]:
# ? rdf.dropna

# Ship Type についてまとめてみた

In [0]:
ShipType=pd.read_csv("AIS_VesselType.csv",index_col=0)

In [274]:
ShipType.columns

Index(['Description'], dtype='object')

In [275]:
ShipType["Description"].unique()

array(['Not available (default)', 'Reserved for future use',
       'Wing in ground (WIG), all ships of this type',
       'Wing in ground (WIG), Hazardous category A',
       'Wing in ground (WIG), Hazardous category B',
       'Wing in ground (WIG), Hazardous category C',
       'Wing in ground (WIG), Hazardous category D',
       'Wing in ground (WIG), Reserved for future use', 'Fishing',
       'Towing', 'Towing: length exceeds 200m or breadth exceeds 25m',
       'Dredging or underwater ops', 'Diving ops', 'Military ops',
       'Sailing', 'Pleasure Craft', 'Reserved',
       'High speed craft (HSC), all ships of this type',
       'High speed craft (HSC), Hazardous category A',
       'High speed craft (HSC), Hazardous category B',
       'High speed craft (HSC), Hazardous category C',
       'High speed craft (HSC), Hazardous category D',
       'High speed craft (HSC), Reserved for future use',
       'High speed craft (HSC), No additional information',
       'Pilot Vessel', '

## 2種類の書き方あり
### １つ目

In [0]:
SelectShip=ShipType[ShipType["Description"].str.contains("Passenger")]

### ２つ目

In [0]:
SelectType=['Military ops','Search and Rescue vessel',
            'Passenger, all ships of this type',
       'Passenger, Hazardous category A',
       'Passenger, Hazardous category B',
       'Passenger, Hazardous category C',
       'Passenger, Hazardous category D',
       'Passenger, Reserved for future use',
       'Passenger, No additional information']
SelectShip=ShipType.query('Description in @SelectType')

In [0]:
iships=SelectShip.index

In [279]:
iships

Int64Index([35, 51, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69], dtype='int64', name='TypeCode')

In [280]:
SelectShip

,Description
TypeCode,
35,Military ops
51,Search and Rescue vessel
60,"Passenger, all ships of this type"
61,"Passenger, Hazardous category A"
62,"Passenger, Hazardous category B"
63,"Passenger, Hazardous category C"
64,"Passenger, Hazardous category D"
65,"Passenger, Reserved for future use"
66,"Passenger, Reserved for future use"


# 限定したShip Type　のデータを抽出します。

In [0]:
dlist=[]
for fi in csvs[:1]:
  rdf = pd.read_csv(fi,usecols=[1,2,3,4,5,6,7,8,10,11,12,13,14,15])
  rdf['datetime'] = pd.to_datetime(rdf['BaseDateTime'])
  rdf=rdf.dropna(subset=["VesselName"])
  rdf.dropna(subset=['VesselType'],inplace=True)
  rdf["VesselType"]=rdf["VesselType"].astype(np.int64)
  rdf.query('VesselType in @iships',inplace=True)
  dlist.append(rdf)

In [0]:
df = pd.concat(dlist, axis=0, ignore_index=True)

In [287]:
df.head()

,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,VesselType,Status,Length,Width,Draft,Cargo,datetime
0,2018-01-01T00:00:01,37.98799,-122.41855,37.0,-189.4,223.0,SOLANO,IMO8988416,60,0.0,41.0,11.0,1.5,NaN,2018-01-01 00:00:01
1,2018-01-01T00:00:02,38.62288,-90.18316,0.0,-99.1,511.0,TOM SAWYER,NaN,60,15.0,25.0,NaN,NaN,NaN,2018-01-01 00:00:02
2,2018-01-01T00:00:05,30.39578,-81.43012,0.3,-118.4,117.0,JEAN RIBAULT,NaN,60,0.0,46.0,17.0,NaN,NaN,2018-01-01 00:00:05
3,2018-01-01T00:00:05,25.79568,-79.95902,17.5,179.9,180.0,FREEDOM OF THE SEAS,IMO9304033,60,0.0,338.0,39.0,8.8,60.0,2018-01-01 00:00:05
4,2018-01-01T00:00:06,29.25520,-89.35855,0.1,-65.9,511.0,MS DEE,IMO8973497,60,0.0,35.0,7.0,NaN,60.0,2018-01-01 00:00:06


In [288]:
len(df)

577241

In [0]:
df["datetime"]=df["datetime"].dt.round("15T")

In [0]:
df["ShipType"]=df["iVesselType"].replace(SelectShip.index, SelectShip["Description"])

In [0]:
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass,datetime,iVesselType,ShipType
0,366950020,2018-01-01T00:00:01,37.98799,-122.41855,37.0,-189.4,223.0,SOLANO,IMO8988416,WDB7930,60.0,0.0,41.0,11.0,1.5,NaN,B,2018-01-01,60,"Passenger, all ships of this type"
1,367552120,2018-01-01T00:00:02,38.62288,-90.18316,0.0,-99.1,511.0,TOM SAWYER,NaN,WDG5963,60.0,15.0,25.0,NaN,NaN,NaN,B,2018-01-01,60,"Passenger, all ships of this type"
2,367475030,2018-01-01T00:00:05,30.39578,-81.43012,0.3,-118.4,117.0,JEAN RIBAULT,NaN,WDG6597,60.0,0.0,46.0,17.0,NaN,NaN,A,2018-01-01,60,"Passenger, all ships of this type"
3,309906000,2018-01-01T00:00:05,25.79568,-79.95902,17.5,179.9,180.0,FREEDOM OF THE SEAS,IMO9304033,C6UZ7,60.0,0.0,338.0,39.0,8.8,60.0,B,2018-01-01,60,"Passenger, all ships of this type"
4,367079560,2018-01-01T00:00:06,29.25520,-89.35855,0.1,-65.9,511.0,MS DEE,IMO8973497,WDC7960,60.0,0.0,35.0,7.0,NaN,60.0,B,2018-01-01,60,"Passenger, all ships of this type"


In [0]:
import plotly.express as px
import datetime as dt

In [0]:
?px.scatter_mapbox

In [0]:
fig = px.scatter_mapbox(df[df["datetime"]<dti.datetime(2018,1,1,0,1,0)],
                     lat="LAT", 
                     lon="LON",
                     color="ShipType",
                     hover_name="VesselName",
                     hover_data=["ShipType","IMO","Length"])
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [0]:
import plotly.graph_objects as go

In [0]:
steplist=df["datetime"].unique

In [0]:
steplist

<bound method Series.unique of 0         2018-01-01
1         2018-01-01
2         2018-01-01
3         2018-01-01
4         2018-01-01
             ...    
2677841   2018-01-06
2677842   2018-01-06
2677843   2018-01-06
2677844   2018-01-06
2677845   2018-01-06
Name: datetime, Length: 2677846, dtype: datetime64[ns]>

In [0]:
fig = px.scatter_geo(df,
                     lat="LAT", 
                     lon="LON",
                     color="ShipType",
                     hover_name="VesselName",
                     hover_data=["ShipType","IMO","Length"],
                     animation_frame="datetime")
fig.show()

KeyboardInterrupt: ignored